In [1]:
import sys
sys.path.insert(0, '/users/ankushroy/iris/src')
print(sys.path)

['/users/ankushroy/iris/src', '/users/ankushroy/iris_1/src/models/tokenizer', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python38.zip', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8/lib-dynload', '', '/users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8/site-packages']


In [2]:
"""
Credits to https://github.com/CompVis/taming-transformers
"""

from dataclasses import dataclass
from typing import Any, Tuple


from einops import rearrange
import torch
import torch.nn as nn

#from dataset import Batch
from lpips import LPIPS
from nets import Encoder, Decoder
from utils import LossWithIntermediateLosses


batch=1
@dataclass
class TokenizerEncoderOutput:
    z: torch.FloatTensor
    z_quantized: torch.FloatTensor
    tokens: torch.LongTensor


class Tokenizer(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int, encoder: Encoder, decoder: Decoder, with_lpips: bool) -> None:
        super().__init__()
        self.vocab_size = vocab_size
        self.encoder = encoder
        self.pre_quant_conv = torch.nn.Conv2d(encoder.config.z_channels, embed_dim, 1)
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.post_quant_conv = torch.nn.Conv2d(embed_dim, decoder.config.z_channels, 1)
        self.decoder = decoder
        self.embedding.weight.data.uniform_(-1.0 / vocab_size, 1.0 / vocab_size)
        self.lpips = LPIPS().eval() if with_lpips else None

    def __repr__(self) -> str:
        return "tokenizer"

    def forward(self, x: torch.Tensor, should_preprocess: bool = False, should_postprocess: bool = False) -> Tuple[torch.Tensor]:
        outputs = self.encode(x, should_preprocess)
        decoder_input = outputs.z + (outputs.z_quantized - outputs.z).detach()
        reconstructions = self.decode(decoder_input, should_postprocess)
        return outputs.z, outputs.z_quantized, reconstructions

    def compute_loss(self, x, **kwargs: Any) -> LossWithIntermediateLosses:
        assert self.lpips is not None
        observations = x
        z, z_quantized, reconstructions = self(observations, should_preprocess=False, should_postprocess=False)

        # Codebook loss. Notes:
        # - beta position is different from taming and identical to original VQVAE paper
        # - VQVAE uses 0.25 by default
        beta = 1.0
        commitment_loss = (z.detach() - z_quantized).pow(2).mean() + beta * (z - z_quantized.detach()).pow(2).mean()

        reconstruction_loss = torch.abs(observations - reconstructions).mean()
        perceptual_loss = torch.mean(self.lpips(observations, reconstructions))

        return LossWithIntermediateLosses(commitment_loss=commitment_loss, reconstruction_loss=reconstruction_loss,perceptual_loss=perceptual_loss)

    def encode(self, x: torch.Tensor, should_preprocess: bool = False) -> TokenizerEncoderOutput:
        if should_preprocess:
            x = self.preprocess_input(x)
        #print("Shape of x:", x.shape)
        shape = x.shape  # (..., C, H, W)
        x = x.view(-1, *shape[-3:])
        #print("Shape of x as (x_view):", x.shape)
        z = self.encoder(x)
        #print("Shape of z:",z.shape)
        z = self.pre_quant_conv(z)
        b, e, h, w = z.shape
        z_flattened = rearrange(z, 'b e h w -> (b h w) e')
        #print("Shape of z_flattend:",z_flattened.shape)
        dist_to_embeddings = torch.sum(z_flattened ** 2, dim=1, keepdim=True) + torch.sum(self.embedding.weight**2, dim=1) - 2 * torch.matmul(z_flattened, self.embedding.weight.t())

        tokens = dist_to_embeddings.argmin(dim=-1)
        #print("Shape of tokens:",tokens.shape)
        z_q = rearrange(self.embedding(tokens), '(b h w) e -> b e h w', b=b, e=e, h=h, w=w).contiguous()
        #print("Shape of z_q:",z_q.shape)
        # Reshape to original
        z = z.reshape(*shape[:-3], *z.shape[1:])
        #print("Shape of z:", z.shape)
        z_q = z_q.reshape(*shape[:-3], *z_q.shape[1:])
        #print("Shape of reshaped z_q:", z_q.shape)
        tokens = tokens.reshape(*shape[:-3], -1)
        #print("Shape of tokens:", tokens.shape)

        return TokenizerEncoderOutput(z, z_q, tokens)

    def decode(self, z_q: torch.Tensor, should_postprocess: bool = False) -> torch.Tensor:
        shape = z_q.shape  # (..., E, h, w)
        z_q = z_q.view(-1, *shape[-3:])
        z_q = self.post_quant_conv(z_q)
        rec = self.decoder(z_q)
        rec = rec.reshape(*shape[:-3], *rec.shape[1:])
        if should_postprocess:
            rec = self.postprocess_output(rec)
        return rec

    @torch.no_grad()
    def encode_decode(self, x: torch.Tensor, should_preprocess: bool = False, should_postprocess: bool = False) -> torch.Tensor:
        z_q = self.encode(x, should_preprocess).z_quantized
        return self.decode(z_q, should_postprocess)

    def preprocess_input(self, x: torch.Tensor) -> torch.Tensor:
        """x is supposed to be channels first and in [0, 1]"""
        return x.mul(2).sub(1)

    def postprocess_output(self, y: torch.Tensor) -> torch.Tensor:
        """y is supposed to be channels first and in [-1, 1]"""
        return y.add(1).div(2)

Pysteps configuration file found at: /users/ankushroy/anaconda3/envs/myenv_exp_1/lib/python3.8/site-packages/pysteps/pystepsrc



In [3]:
from typing import List

import torch
import torch.nn as nn
## Configuration files 
@dataclass
class EncoderDecoderConfig:
    resolution: int
    in_channels: int
    z_channels: int
    ch: int
    ch_mult: List[int]
    num_res_blocks: int
    attn_resolutions: List[int]
    out_ch: int
    dropout: float

In [ ]:
encoder = Encoder(EncoderDecoderConfig(resolution=256,
                                       in_channels=1,
                                        z_channels=1024,
                                        ch=128,
                                        ch_mult= [1, 1, 1, 2, 2, 4],
                                        num_res_blocks= 2,
                                        attn_resolutions= [8],
                                        out_ch= 1,
                                        dropout= 0.0))
decoder=Decoder(EncoderDecoderConfig(resolution=256,
                                       in_channels=1,
                                        z_channels=1024,
                                        ch=128,
                                        ch_mult= [1, 1, 1, 2, 2, 4],
                                        num_res_blocks= 2,
                                        attn_resolutions= [8],  
                                        out_ch= 1,
                                        dropout= 0.0))
vocab_size = 1024 # actual vocabulary size 
embed_dim = 1024  # the desired embedding dimension of the codebook (coebook dim)
tokenizer = Tokenizer(vocab_size, embed_dim, encoder, decoder, with_lpips=True)
print(encoder)
print(decoder)

For 4 x 4 x 512

In [4]:
encoder = Encoder(EncoderDecoderConfig(resolution=128,
                                       in_channels=1,
                                        z_channels=512,
                                        ch=128,
                                        ch_mult= [1, 1, 1, 2, 2, 4],
                                        num_res_blocks= 2,
                                        attn_resolutions= [4],
                                        out_ch= 1,
                                        dropout= 0.0))
decoder=Decoder(EncoderDecoderConfig(resolution=128,
                                       in_channels=1,
                                        z_channels=512,
                                        ch=128,
                                        ch_mult= [1, 1, 1, 2, 2, 4],
                                        num_res_blocks= 2,
                                        attn_resolutions= [4],  
                                        out_ch= 1,
                                        dropout= 0.0))
vocab_size = 1024 # actual vocabulary size 
embed_dim = 512  # the desired embedding dimension of the codebook (coebook dim)
tokenizer = Tokenizer(vocab_size, embed_dim, encoder, decoder, with_lpips=True)
#print(encoder)
#print(decoder)

Tokenizer : shape of latent is (512, 4, 4).


In [5]:
learning_rate=0.0001 # cfg training file 
optimizer_tokenizer = torch.optim.Adam(tokenizer.parameters(), lr=learning_rate)

In [5]:
# Define dataset
import torch
#import sys
#from nuwa_pytorch import VQGanVAE
import h5py
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.ndimage import zoom
def eventGeneration(start_time, obs_time = 3 ,lead_time = 6, time_interval = 30):
    # Generate event based on starting time point, return a list: [[t-4,...,t-1,t], [t+1,...,t+72]]
    # Get the start year, month, day, hour, minute
    year = int(start_time[0:4])
    month = int(start_time[4:6])
    day = int(start_time[6:8])
    hour = int(start_time[8:10])
    minute = int(start_time[10:12])
    #print(datetime(year=year, month=month, day=day, hour=hour, minute=minute))
    times = [(datetime(year, month, day, hour, minute) + timedelta(minutes=time_interval * (x+1))) for x in range(lead_time)]
    lead = [dt.strftime('%Y%m%d%H%M') for dt in times]
    times = [(datetime(year, month, day, hour, minute) - timedelta(minutes=time_interval * x)) for x in range(obs_time)]
    obs = [dt.strftime('%Y%m%d%H%M') for dt in times]
    obs.reverse()
    return lead, obs

from torch.utils.data import Dataset, DataLoader
import h5py
import numpy as np
from torchvision.transforms import ToTensor, Compose, CenterCrop
class radarDataset(Dataset):
    def __init__(self, root_dir, event_times, obs_number = 3, pred_number = 6, transform=None):
        # event_times is an array of starting time t(string)
        # transform is the preprocessing functions
        self.root_dir = root_dir
        self.transform = transform
        self.event_times = event_times
        self.obs_number = obs_number
        self.pred_number = pred_number
    def __len__(self):
        return len(self.event_times)
    def __getitem__(self, idx):
        start_time = str(self.event_times[idx])
        time_list_pre, time_list_obs = eventGeneration(start_time, self.obs_number, self.pred_number)
        output = []
        time_list = time_list_obs + time_list_pre
        #print(time_list)
        for time in time_list:
            year = time[0:4]
            month = time[4:6]
            #path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAC_MFBS_EM_5min_' + time + '_NL.h5'
            path = self.root_dir + year + '/' + month + '/' + 'RAD_NL25_RAP_5min_' + time + '.h5'
            image = np.array(h5py.File(path)['image1']['image_data'])
            #image = np.ma.masked_where(image == 65535, image)
            image = image[264:520,242:498]
            image = zoom(image, (128 / image.shape[0], 128 / image.shape[1]))
            image[image == 65535] = 0
            image = image.astype('float32')
            image = image/100*12
            image = np.clip(image, 0, 128)
            image = image/40
            #image = 2*image-1 #normalize to [-1,1]
            output.append(image)
        output = torch.permute(torch.tensor(np.array(output)), (1, 2, 0))
        output = self.transform(np.array(output))
        return output
#root_dir = '/users/hbi/data/RAD_NL25_RAC_MFBS_EM_5min/'
#dataset = radarDataset(root_dir, ["200808031600"], transform = Compose([ToTensor(),CenterCrop(256)]))

In [6]:
# develop dataset
from torch.cuda.amp import autocast
#from torch.autograd import Variable
import pandas as pd
root_dir = '/home/hbi/RAD_NL25_RAP_5min/' 
batch_size=1

df_train = pd.read_csv('/users/ankushroy/taming-transformers/training_Delfland08-14_20.csv', header = None)
event_times = df_train[0].to_list()
dataset_train = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_train_s = pd.read_csv('/users/ankushroy/taming-transformers/training_Delfland08-14.csv', header = None)
event_times = df_train_s[0].to_list()
dataset_train_del = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_test = pd.read_csv('/users/ankushroy/taming-transformers/testing_Delfland18-20.csv', header = None)
event_times = df_test[0].to_list()
dataset_test = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_vali = pd.read_csv('/users/ankushroy/taming-transformers/validation_Delfland15-17.csv', header = None)
event_times = df_vali[0].to_list()
dataset_vali = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))

df_train_aa = pd.read_csv('/users/ankushroy/taming-transformers/training_Aa08-14.csv', header = None)
event_times = df_train_aa[0].to_list()
dataset_train_aa = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))  

df_train_dw = pd.read_csv('/users/ankushroy/taming-transformers/training_Dwar08-14.csv', header = None)
event_times = df_train_dw[0].to_list()
dataset_train_dw = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))    

df_train_re = pd.read_csv('/users/ankushroy/taming-transformers/training_Regge08-14.csv', header = None)
event_times = df_train_re[0].to_list()
dataset_train_re = radarDataset(root_dir, event_times, transform = Compose([ToTensor()]))   

data_list = [dataset_train_aa, dataset_train_dw, dataset_train_del, dataset_train_re]
train_aadedwre = torch.utils.data.ConcatDataset(data_list)

print(len(train_aadedwre), len(dataset_test), len(dataset_vali))
loaders = { 'train' :DataLoader(train_aadedwre, batch_size, shuffle=True, num_workers=8),
            'test' :DataLoader(dataset_test, batch_size, shuffle=False, num_workers=8), 
           'valid' :DataLoader(dataset_vali, batch_size, shuffle=False, num_workers=8),
          
          'train_aa5' :DataLoader(dataset_train_aa, batch_size, shuffle=False, num_workers=8),
          'train_dw5' :DataLoader(dataset_train_dw, batch_size, shuffle=False, num_workers=8),
          'train_del5' :DataLoader(dataset_train_del, batch_size, shuffle=True, num_workers=8),
          'train_re5' :DataLoader(dataset_train_re, batch_size, shuffle=False, num_workers=8),
          }


30632 3493 3560


In [7]:
# Initializing Device as cuda:0
cuda_version = torch.version.cuda
print("CUDA version:", cuda_version)
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print("PyTorch version:", torch.__version__)

CUDA version: 11.3
True
cuda:0
PyTorch version: 1.11.0


In [ ]:
##Training loop VQVAE 
import numpy as np
from collections import defaultdict
loss_total_step=0.0
intermediate_losses = defaultdict(float)
num_epochs=80
device = torch.device("cuda:0")  # Specify the GPU device
epoch_start=0
tokenizer.to(device)
all_metrics = []  # List to store metrics for all epochs

for epoch in range(epoch_start,num_epochs):
    save_epoch = epoch in [0,2,4,9,19,29,39,49,59,79]
    #save_epoch = epoch in [2]
    loss_total_epoch=0.0
    #optimizer_tokenizer.zero_grad()
    intermediate_losses = {}
    print("epoch {}".format(epoch))
    for i, images in enumerate(loaders['train']):
        loss_total_step=0.0
        image = images[0]
        image = image.unsqueeze(1)
        image = image[3:4, :, :, :]
        input_image= image.to(device)  # batch x
        #print(input_image.size())
        encoder_output = tokenizer.encode(input_image)
        losses=tokenizer.compute_loss(input_image)
        
        loss_total_step += losses.loss_total 
        loss_total_epoch += loss_total_step.item()/64
        if (i+1) % 64 == 0:
            (loss_total_step/64).backward()
            optimizer_tokenizer.step()
            optimizer_tokenizer.zero_grad()
            
            print("Losses: Total = {:.4f}".format(loss_total_step.item()))
            #loss_total_step=0.0
        

        for loss_name, loss_value in losses.intermediate_losses.items():
                intermediate_losses[f"{str(tokenizer)}/train/{loss_name}"] = loss_value/64
    
        
    metrics = {f'{str(Tokenizer)}/train/total_loss': loss_total_epoch, **intermediate_losses}
    print("Epoch {}: Total Loss = {:.4f}".format(epoch, metrics[f'{str(Tokenizer)}/train/total_loss']))

    if save_epoch:
        torch.save({
        'model_state_dict': tokenizer.state_dict(),
        'optimizer_state_dict': optimizer_tokenizer.state_dict(),
        }, '/space/ankushroy/Tokeniser_exp2_1024_embdim_8/vqvae_checkpoint_epoch{}'.format(epoch+1))

    all_metrics.append(metrics)  # Save metrics for the current epoch to the list


# Convert the metrics list to a NumPy array
metrics_array = np.array(all_metrics)

# Save the metrics array as a NumPy file
np.save('/space/ankushroy/Tokeniser_exp2_1024_embdim_8/metrices.npy', metrics_array)
# Print all errors from each epoch's metrics dictionary
# print(all_metrics)


In [9]:
#torch.cuda.empty_cache()
import time
from pysteps.verification.detcatscores import det_cat_fct
from pysteps.verification.detcontscores import det_cont_fct
from pysteps.verification.spatialscores import intensity_scale
from pysteps.visualization import plot_precip_field
threshold = 0.0
device = 'cuda:0'
pcc_average = 0
tokenizer=tokenizer.to(device)
tokenizer.eval()
for i, images in enumerate(loaders['test']):
    if i<0:continue
    if i>=5:break

    
    checkpoint = torch.load('/space/ankushroy/last.pt', map_location=device)
    tokenizer.load_state_dict(checkpoint['model_state_dict'])
    #optimizer_tokenizer.load_state_dict(checkpoint['optimizer_state_dict'])
    image = images[0]
    image = image.unsqueeze(1)
    image = image[4:5, :, :, :]
    input_image= image.to(device)  # batch x
        #print(input_image.size())
    a_r = tokenizer.encode_decode(input_image)
        #checkpoint = torch.load('/users/zboucher/iris/src/models/tokenizer/vqvae_epoch70', map_location = 'cpu')
        #vae.load_state_dict(checkpoint)
        #a_r2 = vae(a)

    for t in range(1):
        a_display = input_image[t,0,:,:].to('cpu').detach().numpy()*40
        a_r_display = a_r[t,0,:,:].to('cpu').detach().numpy()*40
        
        a_r_display[a_r_display < threshold] = 0.0
        
        scores_cat1 = det_cat_fct(a_r_display, a_display, 1)
        scores_cat2 = det_cat_fct(a_r_display, a_display, 2)
        scores_cat8 = det_cat_fct(a_r_display, a_display, 8)
        scores_cont = det_cont_fct(a_r_display, a_display, thr=0.1)
        
        scores_spatial = intensity_scale(a_r_display, a_display, 'FSS', 0.1, [1,10,20,30])
        pcc_average += float(np.around(scores_cont['corr_p'],3))
        if True:
            print('MSE:', np.around(scores_cont['MSE'],3), 
                    'MAE:', np.around(scores_cont['MAE'],3), 
                    'PCC:', np.around(scores_cont['corr_p'],3),'\n', 
                    'CSI(1mm):', np.around(scores_cat1['CSI'],3), # CSI: TP/(TP+FP+FN)
                    'CSI(2mm):', np.around(scores_cat2['CSI'],3),
                    'CSI(8mm):', np.around(scores_cat8['CSI'],3),'\n',
                    'ACC(1mm):', np.around(scores_cat1['ACC'],3), # ACC: (TP+TF)/(TP+TF+FP+FN)
                    'ACC(2mm):', np.around(scores_cat2['ACC'],3),
                    'ACC(8mm):', np.around(scores_cat8['ACC'],3),'\n',
                    'FSS(1km):', np.around(scores_spatial[0][0],3),
                    'FSS(10km):', np.around(scores_spatial[1][0],3),
                    'FSS(20km):', np.around(scores_spatial[2][0],3),
                    'FSS(30km):', np.around(scores_spatial[3][0],3)
                    )  
            plt.figure(figsize=(16, 4))
            plt.subplot(131)
            plot_precip_field(a_display, title="Input")
            plt.subplot(132)
            plot_precip_field(a_r_display, title="Reconstruction")
            
            plt.tight_layout()
            plt.show()
print('pcc_average:', pcc_average/i)

KeyError: 'model_state_dict'

In [11]:
checkpoint = torch.load('/space/ankushroy/last.pt', map_location=device)
print(checkpoint.keys())

odict_keys(['tokenizer.encoder.conv_in.weight', 'tokenizer.encoder.conv_in.bias', 'tokenizer.encoder.down.0.block.0.norm1.weight', 'tokenizer.encoder.down.0.block.0.norm1.bias', 'tokenizer.encoder.down.0.block.0.conv1.weight', 'tokenizer.encoder.down.0.block.0.conv1.bias', 'tokenizer.encoder.down.0.block.0.norm2.weight', 'tokenizer.encoder.down.0.block.0.norm2.bias', 'tokenizer.encoder.down.0.block.0.conv2.weight', 'tokenizer.encoder.down.0.block.0.conv2.bias', 'tokenizer.encoder.down.0.block.1.norm1.weight', 'tokenizer.encoder.down.0.block.1.norm1.bias', 'tokenizer.encoder.down.0.block.1.conv1.weight', 'tokenizer.encoder.down.0.block.1.conv1.bias', 'tokenizer.encoder.down.0.block.1.norm2.weight', 'tokenizer.encoder.down.0.block.1.norm2.bias', 'tokenizer.encoder.down.0.block.1.conv2.weight', 'tokenizer.encoder.down.0.block.1.conv2.bias', 'tokenizer.encoder.down.0.downsample.conv.weight', 'tokenizer.encoder.down.0.downsample.conv.bias', 'tokenizer.encoder.down.1.block.0.norm1.weight', '

In [14]:
checkpoint_1 = torch.load('/space/ankushroy/Tokeniser_Chkpts_recon_tokens_256/vqvae_epoch5', map_location=device)
print(checkpoint_1.keys())

dict_keys(['state', 'param_groups'])


: 

Training the Model from 250th Epoch to 350th Epoch


In [ ]:
import numpy as np
from collections import defaultdict
checkpoint_path = '/space/ankushroy/Tokeniser_exp2_1024_embdim_9/vqvae_checkpoint_epoch300'
checkpoint = torch.load(checkpoint_path,map_location=device)
tokenizer.load_state_dict(checkpoint['model_state_dict'])
optimizer_tokenizer.load_state_dict(checkpoint['optimizer_state_dict'])
loss_total_step=0.0
intermediate_losses = defaultdict(float)

#device = torch.device("cuda:1")  # Specify the GPU device
tokenizer.to(device)

epoch_start=300
num_epochs=400

all_metrics = []  # List to store metrics for all epochs

for epoch in range(epoch_start,num_epochs):
    save_epoch = epoch in [304,309,319,329,339,349,359,369,379,389,399]
    
    loss_total_epoch=0.0
    optimizer_tokenizer.zero_grad()
    intermediate_losses = {}
    print("epoch {}".format(epoch))
    for i, images in enumerate(loaders['train']):
        loss_total_step=0.0
        image = images[0]
        image = image.unsqueeze(1)
        image = image[3:4, :, :, :]
        input_image= image.to(device)  # batch x
        #print(input_image.size())
        encoder_output = tokenizer.encode(input_image)
        losses=tokenizer.compute_loss(input_image)
        
        loss_total_step += losses.loss_total 
        loss_total_epoch += loss_total_step.item()/64
        if (i+1) % 64 == 0:
            (loss_total_step/64).backward()
            optimizer_tokenizer.step()
            optimizer_tokenizer.zero_grad()
            
            print("Losses: Total = {:.4f}".format(loss_total_step.item()))
            #loss_total_step=0.0
        

        for loss_name, loss_value in losses.intermediate_losses.items():
                intermediate_losses[f"{str(tokenizer)}/train/{loss_name}"] = loss_value/64
    
        
    metrics = {f'{str(Tokenizer)}/train/total_loss': loss_total_epoch, **intermediate_losses}
    print("Epoch {}: Total Loss = {:.4f}".format(epoch, metrics[f'{str(Tokenizer)}/train/total_loss']))

    if save_epoch:
        torch.save({
        'model_state_dict': tokenizer.state_dict(),
        'optimizer_state_dict': optimizer_tokenizer.state_dict(),
        }, '/space/ankushroy/Tokeniser_exp2_1024_embdim_10/vqvae_checkpoint_epoch{}'.format(epoch+1))

    all_metrics.append(metrics)  # Save metrics for the current epoch to the list


# Convert the metrics list to a NumPy array
metrics_array = np.array(all_metrics)

# Save the metrics array as a NumPy file
np.save('/space/ankushroy/Tokeniser_exp2_1024_embdim_10/metrices.npy', metrics_array)
# Print all errors from each epoch's metrics dictionary
# print(all_metrics)

epoch 300
Losses: Total = 0.1009
Losses: Total = 0.0462
Losses: Total = 0.1195
Losses: Total = 0.0700
Losses: Total = 0.0759
Losses: Total = 0.0690
Losses: Total = 0.1020
Losses: Total = 0.0943
Losses: Total = 0.1173
Losses: Total = 0.1491
Losses: Total = 0.0309
Losses: Total = 0.0562
Losses: Total = 0.0422
Losses: Total = 0.1920
Losses: Total = 0.1400
Losses: Total = 0.1516
Losses: Total = 0.0836
Losses: Total = 0.1770
Losses: Total = 0.1042
Losses: Total = 0.1125
Losses: Total = 0.1228
Losses: Total = 0.1150
Losses: Total = 0.0347
Losses: Total = 0.0660
Losses: Total = 0.1127
Losses: Total = 0.1214
Losses: Total = 0.0676
Losses: Total = 0.0438
Losses: Total = 0.0963
Losses: Total = 0.0570
Losses: Total = 0.1138
Losses: Total = 0.0985
Losses: Total = 0.1026
Losses: Total = 0.0671
Losses: Total = 0.0948
Losses: Total = 0.0861
Losses: Total = 0.1576
Losses: Total = 0.0763
Losses: Total = 0.0331
Losses: Total = 0.0947
Losses: Total = 0.0993
Losses: Total = 0.1013
Losses: Total = 0.1060
L

Losses: Total = 0.0573
Losses: Total = 0.0738
Losses: Total = 0.0652
Losses: Total = 0.1723
Losses: Total = 0.0276
Losses: Total = 0.1485
Losses: Total = 0.0428
Losses: Total = 0.0388
Losses: Total = 0.0864
Losses: Total = 0.0863
Losses: Total = 0.0558
Losses: Total = 0.0529
Losses: Total = 0.0782
Losses: Total = 0.0523
Losses: Total = 0.0953
Losses: Total = 0.0563
Losses: Total = 0.1353
Losses: Total = 0.1438
Losses: Total = 0.2012
Losses: Total = 0.1101
Losses: Total = 0.1080
Losses: Total = 0.0902
Losses: Total = 0.1066
Losses: Total = 0.1126
Losses: Total = 0.1022
Losses: Total = 0.0912
Losses: Total = 0.1040
Losses: Total = 0.1132
Losses: Total = 0.1177
Losses: Total = 0.1228
Losses: Total = 0.0896
Losses: Total = 0.1623
Losses: Total = 0.0770
Losses: Total = 0.0486
Losses: Total = 0.1869
Losses: Total = 0.1175
Losses: Total = 0.0792
Losses: Total = 0.0441
Losses: Total = 0.0163
Losses: Total = 0.1671
Losses: Total = 0.0415
Losses: Total = 0.1188
Losses: Total = 0.1243
Losses: Tot

Losses: Total = 0.0277
Losses: Total = 0.1090
Losses: Total = 0.0364
Losses: Total = 0.1789
Losses: Total = 0.1110
Losses: Total = 0.1385
Losses: Total = 0.0657
Losses: Total = 0.0706
Losses: Total = 0.0635
Losses: Total = 0.0349
Losses: Total = 0.1264
Losses: Total = 0.0888
Losses: Total = 0.0979
Losses: Total = 0.0408
Losses: Total = 0.1470
Losses: Total = 0.0717
Losses: Total = 0.1345
Losses: Total = 0.0498
Losses: Total = 0.0752
Losses: Total = 0.1618
Losses: Total = 0.1089
Losses: Total = 0.1735
Losses: Total = 0.0663
Losses: Total = 0.1090
Losses: Total = 0.0441
Losses: Total = 0.0790
Losses: Total = 0.1012
Losses: Total = 0.1156
Losses: Total = 0.1092
Losses: Total = 0.1640
Losses: Total = 0.0441
Losses: Total = 0.1068
Losses: Total = 0.1051
Losses: Total = 0.1708
Losses: Total = 0.0303
Losses: Total = 0.0319
Losses: Total = 0.1047
Losses: Total = 0.0546
Losses: Total = 0.1235
Losses: Total = 0.1170
Losses: Total = 0.1158
Losses: Total = 0.0460
Losses: Total = 0.0899
Losses: Tot

Losses: Total = 0.1092
Losses: Total = 0.0628
Losses: Total = 0.1323
Losses: Total = 0.0798
Losses: Total = 0.1454
Losses: Total = 0.0452
Losses: Total = 0.1584
Losses: Total = 0.0975
Losses: Total = 0.1772
Losses: Total = 0.1403
Losses: Total = 0.1620
Losses: Total = 0.1104
Losses: Total = 0.0657
Losses: Total = 0.1250
Losses: Total = 0.0904
Losses: Total = 0.0938
Losses: Total = 0.0743
Losses: Total = 0.1569
Losses: Total = 0.0934
Losses: Total = 0.0304
Losses: Total = 0.0458
Losses: Total = 0.0014
Losses: Total = 0.0872
Losses: Total = 0.1333
Losses: Total = 0.1053
Losses: Total = 0.1058
Losses: Total = 0.1117
Losses: Total = 0.0822
Losses: Total = 0.0411
Losses: Total = 0.0112
Losses: Total = 0.0618
Losses: Total = 0.1405
Losses: Total = 0.0247
Losses: Total = 0.1575
Losses: Total = 0.0646
Losses: Total = 0.0414
Losses: Total = 0.2146
Losses: Total = 0.0648
Losses: Total = 0.1179
Losses: Total = 0.1408
Losses: Total = 0.0294
Losses: Total = 0.1781
Losses: Total = 0.0985
Losses: Tot

Losses: Total = 0.0222
Losses: Total = 0.0792
Losses: Total = 0.1451
Losses: Total = 0.1117
Losses: Total = 0.0620
Losses: Total = 0.0628
Losses: Total = 0.1337
Losses: Total = 0.0077
Losses: Total = 0.1311
Losses: Total = 0.0621
Losses: Total = 0.0270
Epoch 302: Total Loss = 45.5354
epoch 303
Losses: Total = 0.0224
Losses: Total = 0.0671
Losses: Total = 0.0752
Losses: Total = 0.1226
Losses: Total = 0.0971
Losses: Total = 0.0589
Losses: Total = 0.1082
Losses: Total = 0.0778
Losses: Total = 0.1712
Losses: Total = 0.1602
Losses: Total = 0.0921
Losses: Total = 0.1260
Losses: Total = 0.1585
Losses: Total = 0.0480
Losses: Total = 0.0400
Losses: Total = 0.0848
Losses: Total = 0.2292
Losses: Total = 0.1083
Losses: Total = 0.0906
Losses: Total = 0.2026
Losses: Total = 0.0227
Losses: Total = 0.0742
Losses: Total = 0.1252
Losses: Total = 0.0870
Losses: Total = 0.0681
Losses: Total = 0.0951
Losses: Total = 0.0437
Losses: Total = 0.1733
Losses: Total = 0.1435
Losses: Total = 0.1285
Losses: Total =

...

Losses: Total = 0.0368
Losses: Total = 0.1005
Losses: Total = 0.0961
Losses: Total = 0.0635
Losses: Total = 0.0578
Losses: Total = 0.0111
Losses: Total = 0.1550
Losses: Total = 0.1829
Losses: Total = 0.0865
Losses: Total = 0.1223
Losses: Total = 0.0788
Losses: Total = 0.1720
Losses: Total = 0.1364
Losses: Total = 0.0489
Losses: Total = 0.1651
Losses: Total = 0.0830
Losses: Total = 0.1212
Losses: Total = 0.0361
Losses: Total = 0.0767
Losses: Total = 0.0689
Losses: Total = 0.1272
Losses: Total = 0.1132
Losses: Total = 0.1419
Losses: Total = 0.1085
Losses: Total = 0.0804
Losses: Total = 0.0727
Losses: Total = 0.1164
Losses: Total = 0.0357
Losses: Total = 0.1013
Losses: Total = 0.0848
Losses: Total = 0.1152
Losses: Total = 0.1575
Losses: Total = 0.0104
Losses: Total = 0.0700
Losses: Total = 0.0589
Losses: Total = 0.1322
Losses: Total = 0.0889
Losses: Total = 0.0628
Losses: Total = 0.1266
Losses: Total = 0.1290
Losses: Total = 0.0575
Losses: Total = 0.1111
Losses: Total = 0.0431
Losses: Tot

Losses: Total = 0.0813
Losses: Total = 0.0608
Losses: Total = 0.1829
Losses: Total = 0.1439
Losses: Total = 0.1144
Losses: Total = 0.0938
Losses: Total = 0.0243
Losses: Total = 0.1351
Losses: Total = 0.0583
Losses: Total = 0.0765
Losses: Total = 0.1093
Losses: Total = 0.0750
Losses: Total = 0.1031
Losses: Total = 0.0767
Losses: Total = 0.0693
Losses: Total = 0.0820
Losses: Total = 0.0949
Losses: Total = 0.0321
Losses: Total = 0.0689
Losses: Total = 0.0713
Losses: Total = 0.2037
Losses: Total = 0.1022
Losses: Total = 0.0770
Losses: Total = 0.1211
Losses: Total = 0.0694
Losses: Total = 0.0781
Losses: Total = 0.0623
Losses: Total = 0.1229
Losses: Total = 0.1134
Losses: Total = 0.0466
Losses: Total = 0.1149
Losses: Total = 0.0583
Losses: Total = 0.1079
Losses: Total = 0.1149
Losses: Total = 0.0919
Losses: Total = 0.1394
Losses: Total = 0.0351
Losses: Total = 0.0617
Losses: Total = 0.1124
Losses: Total = 0.1022
Losses: Total = 0.1313
Losses: Total = 0.0766
Losses: Total = 0.0796
Losses: Tot

Losses: Total = 0.0866
Losses: Total = 0.1544
Losses: Total = 0.1105
Losses: Total = 0.0600
Losses: Total = 0.0966
Losses: Total = 0.1196
Losses: Total = 0.1678
Losses: Total = 0.0711
Losses: Total = 0.0739
Losses: Total = 0.2078
Losses: Total = 0.1504
Losses: Total = 0.1072
Losses: Total = 0.1363
Losses: Total = 0.0801
Losses: Total = 0.0623
Losses: Total = 0.0334
Losses: Total = 0.0873
Losses: Total = 0.0315
Losses: Total = 0.1131
Losses: Total = 0.1101
Losses: Total = 0.0930
Losses: Total = 0.1277
Losses: Total = 0.0243
Losses: Total = 0.1122
Losses: Total = 0.0514
Losses: Total = 0.0579
Losses: Total = 0.1147
Losses: Total = 0.1452
Losses: Total = 0.2116
Losses: Total = 0.0961
Losses: Total = 0.0258
Losses: Total = 0.1000
Losses: Total = 0.1405
Losses: Total = 0.0327
Losses: Total = 0.1450
Losses: Total = 0.1058
Losses: Total = 0.1273
Losses: Total = 0.0244
Losses: Total = 0.0540
Losses: Total = 0.0512
Losses: Total = 0.0084
Losses: Total = 0.0510
Losses: Total = 0.1017
Losses: Tot

Losses: Total = 0.0812
Losses: Total = 0.0935
Losses: Total = 0.0778
Losses: Total = 0.0537
Losses: Total = 0.0743
Losses: Total = 0.1695
Losses: Total = 0.0656
Losses: Total = 0.0873
Losses: Total = 0.0913
Losses: Total = 0.0868
Losses: Total = 0.0189
Losses: Total = 0.0353
Losses: Total = 0.0662
Losses: Total = 0.0977
Losses: Total = 0.0606
Losses: Total = 0.0725
Losses: Total = 0.1184
Losses: Total = 0.1454
Losses: Total = 0.1450
Losses: Total = 0.0475
Losses: Total = 0.1102
Losses: Total = 0.0587
Losses: Total = 0.1238
Losses: Total = 0.1152
Losses: Total = 0.0950
Losses: Total = 0.1409
Losses: Total = 0.1229
Losses: Total = 0.0773
Losses: Total = 0.1031
Losses: Total = 0.0729
Losses: Total = 0.0446
Losses: Total = 0.0647
Losses: Total = 0.0864
Losses: Total = 0.1036
Losses: Total = 0.0343
Losses: Total = 0.1197
Losses: Total = 0.0644
Losses: Total = 0.0123
Losses: Total = 0.0818
Losses: Total = 0.0965
Losses: Total = 0.1371
Losses: Total = 0.0965
Losses: Total = 0.1135
Losses: Tot

Losses: Total = 0.0610
Losses: Total = 0.0566
Losses: Total = 0.0629
Losses: Total = 0.1349
Losses: Total = 0.1418
Losses: Total = 0.0788
Losses: Total = 0.0612
Losses: Total = 0.1495
Losses: Total = 0.1044
Losses: Total = 0.1146
Losses: Total = 0.1117
Losses: Total = 0.0810
Losses: Total = 0.0618
Losses: Total = 0.0160
Losses: Total = 0.1937
Losses: Total = 0.0587
Losses: Total = 0.1044
Losses: Total = 0.1109
Losses: Total = 0.0337
